In [ ]:
# 导入需要使用的包 
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import webbrowser
import random
import copy
import cv2
import re

from sklearn.cluster import DBSCAN
from matplotlib import cm
from tqdm import tqdm
from tkinter import *
from tkinter import ttk

from PIL import ImageFilter
from PIL import ImageOps
from PIL import ImageTk
from PIL import Image

In [ ]:
# CNN 解释器
# 用于解释 CNN 预测的关键决策依据

def Explain(pixels,model,index,Scores, TEXT):
    # 存放每一个像素的位置坐标 (用以画散点图)
    X = []
    # 存放每一个像素的 “重要性” (用以画散点图)
    S = []
    # 遮盖半径 相当于遮盖 (size*2+1)**2 的面积
    size = 2
    # 构建进度条
    pbar = tqdm(range(size,28-size),ncols=45)

    for row in pbar:
        TEXT[0].delete(1.0, END)
        TEXT[0].insert(END, '正在解释CNN决策依据 请稍等\n'+str(pbar))
        TEXT[0].update()

        for col in range(size,28-size):
            # 复制出一个同样的图片 temp
            temp = copy.deepcopy(pixels)
            # row,col 坐标为中心进行像素遮盖，这里采用置零法
            temp[row-size:row+size+1,col-size:col+size+1] = 0
            # 重新预测被遮盖的图(复制的temp)
            p = model.predict(temp.reshape(-1,28,28,1))
            X.append([row,col])
            # 看看预测结果的改变程度, 保存在S中
            # 分析：
            # 如果被遮盖的地方，不影响预测结果，那就说明这个地方不是模型的决策关键
            S.append(abs((p[0][index]-Scores)))
    
    # print('INFO',index)
    TEXT[0].delete(1.0, END)
    TEXT[0].insert(END, '正在解释CNN决策依据 请稍等\n'+str(pbar))
    TEXT[0].update()

    X = np.asarray(X)
    S = np.asarray(S)

    S = (S - np.min(S))/(np.max(S)- np.min(S))  # 同一个图上 重要性归一化
    S = (S - np.mean(S))/np.var(S) # 同一个图上 重要性归一化


    TEXT[0].insert(END, '\n---------------------------------------------')
    TEXT[0].insert(END, '\n >90% 重要性 : '+str(round(len(S[S>0.90])/len(S)*100, 4))+'%')
    TEXT[0].insert(END, '\n >80% 重要性 : '+str(round(len(S[S>0.80])/len(S)*100, 4))+'%')
    TEXT[0].insert(END, '\n >70% 重要性 : '+str(round(len(S[S>0.70])/len(S)*100, 4))+'%')
    TEXT[0].insert(END, '\n >50% 重要性 : '+str(round(len(S[S>0.50])/len(S)*100, 4))+'%')

    # 画图保存
    plt.figure(figsize=(28, 28))
    plt.imshow(pixels, cmap='Greys')
    plt.scatter(X[:,1],X[:,0],c=S,s=S*250,alpha=0.8,cmap='Wistia')
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.axis('off')
    plt.savefig("images/split/exp.png",pad_inches = 0,bbox_inches = 'tight',dpi=20)

In [ ]:
def ImageReader(argv):
    # 读取图片
    img = Image.open(argv).convert('L')
    # 获取图片的长和宽
    width = int(float(img.size[0]))
    height = int(float(img.size[1]))
    # 获取像素值
    pixelValue = list(img.getdata())
    # 归一化处理
    pixelValue = [(255 - x) * 1.0 / 255.0 for x in pixelValue]

    return [pixelValue, height, width]

In [ ]:
def ShowDetectionBoxes (image_file, results, detected_image_file=None, verbose=True):
    # 显示检测框
    image = cv2.imread(image_file)
    img_cp = image.copy()

    # 画框
    for i in range(len(results)):
        x = int(results[i][1])
        y = int(results[i][2])
        w = int(results[i][3]) // 2
        h = int(results[i][4]) // 2
        c = results[i][6]
        if verbose:
            # 中心坐标 + 宽高box(x, y, w, h) -> xmin = x - w / 2 -> 左上 + 右下box(xmin, ymin, xmax, ymax)
            cv2.rectangle(img_cp, (x - w, y - h), (x + w, y + h),c, 2)

            # 在边界框上显示类别、分数(类别置信度)
            cv2.rectangle(img_cp, (x - w, y - h - 20), (x + w, y - h), c, -1) # puttext函数的背景
            cv2.putText(img_cp, results[i][0] + ':%.2f' % results[i][5], (x - w + 5, y - h - 7),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
                        
    # 保存绘制好检测框的图片
    if detected_image_file:
        cv2.imwrite(detected_image_file, img_cp)

In [ ]:
# 构建模型类
class Model:
    def __init__(self):
        # 导入先前已经预测好的模型
        self.model = tf.keras.models.load_model('models/model.h5')
    
    # 预测函数
    # TEXT 就是一个控件，用以显示解释CNN时候的进度条
    def Predict(self, image,):
        # 获取图片的：像素值、高度、宽度
        pixelValue, height, width = ImageReader(image)
        # 存放为一个数组
        pixelValueArray = [pixelValue]
        # print('INFO',len(array[0]))
        # 构建一个全0矩阵 size = (height,width)
        imageArray = [[0 for d in range(width)] for y in range(height)]
        
        # 将归一化的像素值一个一个的赋值给构造的全0矩阵 imageArray
        k = 0
        for i in range(height):
            for j in range(width):
                imageArray[i][j] = pixelValueArray[0][k]
                k = k + 1
        
        # dataset 用以存储像素值大于0.9的像素坐标，用以 无监督聚类
        dataset = []
        for i in range(len(imageArray)):
            for j in range(len(imageArray[0])):
                if imageArray[i][j] > 0.9:
                    dataset.append([-i,j])
        dataset = np.asarray(dataset)
        # 使用 DBSCAN 进行所有像素点的无监督聚类
        y_pred = DBSCAN(eps = 10).fit_predict(dataset)
        plt.scatter(dataset[:, 0], dataset[:, 1], c=y_pred)

        # 因为聚类后是没有顺序的，但是我们希望从左到右，一个一个的预测
        order = []
        for oneClass in list(set(y_pred)):
            X_i = dataset[y_pred == oneClass]
            order.append([np.mean(X_i,axis=0)[1],oneClass])
        order.sort()
        # 根据笔画的投影 进行排序，确定所有“符号”的左右顺序

        # 保存每一个被识别出来的 数字 或者 运算符
        for position, oneClass in order:
            X_i = dataset[y_pred == oneClass]
            plt.figure(figsize=(28,28),)
            plt.axis('equal')
            ax=plt.gca()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            plt.axis('off')
            plt.margins(0, 0)
            plt.scatter(X_i[:, 1], X_i[:, 0], s=10000 ,c='black')
            plt.savefig("images/split/"+str(oneClass)+".png",pad_inches = 0,dpi=10)
        
        
        imageArray = np.array(imageArray).reshape(height,width)
        imageArray = (1-imageArray)*255
        im = Image.fromarray(imageArray)
        im = im.convert('L')  # 这样才能转为灰度图，如果是彩色图则改L为‘RGB’
        im.save('images/current.png')

        # 预测 每一个被识别出来的 数字 或者 运算符
        predictions = []
        Scores = []
        for position, oneClass in order:
            splitImage = ImageOps.invert(Image.open("images/split/"+str(oneClass)+".png").convert('L'))
            splitImage = np.array(splitImage.resize(size=(28,28)))
            splitImage = splitImage / 255.0
            splitImage = splitImage.reshape(1, 28, 28, 1)
            scores = self.model.predict(np.array(splitImage))

            number = 0
            bestScore = -1
            prediction = -1
            for score in scores[0]:
                if score > bestScore:
                    bestScore = score
                    prediction = number
                number += 1
            
            predictions.append(prediction)
            Scores.append(np.max(scores[0]))
        
        # 选择一个color map进行颜色选择
        viridis = cm.get_cmap('Set2',len(list(set(y_pred))))
        colors = [viridis(i*1/len(list(set(y_pred)))) for i in range(len(list(set(y_pred))))]

        # result 用以存储 画框 的的信息： 左上角x坐标、左上角y坐标、高度、宽度、颜色、置信度
        results = []
        count = 0
        for position, oneClass in order:
            X_i = dataset[y_pred == oneClass]
            x_min = abs(np.min(X_i[:,1]))
            y_min = abs(np.min(X_i[:,0]))
            x_max = abs(np.max(X_i[:,1]))
            y_max = abs(np.max(X_i[:,0]))
            x_cen = abs(x_max+x_min)/2
            y_cen = abs(y_max+y_min)/2
            w = abs(x_max-x_min)
            h = abs(y_max-y_min)
            string = ''
            if predictions[count] > 9:
                if predictions[count] == 10:
                    string += 'add'
                elif predictions[count] == 11:
                    string += 'sub'
                elif predictions[count] == 12:
                    string += 'mul'
                elif predictions[count] == 13:
                    string += 'div'
                elif predictions[count] == 14:
                    string += '('
                elif predictions[count] == 15:
                    string += ')'
            else:
                string += str(predictions[count])
                
            results.append((string, x_cen, y_cen, w, h, Scores[count],tuple([int(colors[count][i]*255) for i in range(3)])))
            count += 1
            
        # 绘制检测框
        ShowDetectionBoxes(image_file='images/current.png',results=results,detected_image_file='images/current.png')
        
        return predictions, order, Scores

In [ ]:
# 构建计算类
class Calculator(object):
    def __init__(self):
        # 设置运算符优先级
        self.Operator={
            '(':4,
            ')':3,
            '×':2,
            '÷':2,
            '+':1,
            '-':1,
        }

    def operate(self, operandA, operandB, operator):
        # 设置运算符对应的计算方式
        if operator == '+':
            return operandA + operandB
        elif operator == '-':
            return operandA - operandB
        elif operator == '×':
            return operandA * operandB
        elif operator == '÷':
            return operandA/operandB
        else:
            print("运算符有误，请确认！")
            return None

    def isFormula(self, strs):
        # 判断是否是正确的算式
        temp = []
        strs = strs.replace(' ', '')
        strs = strs.replace('×', '*')
        strs = strs.replace('÷', '/')
        # print('INFO:',strs)
        for i in strs:
            if i is '(':
                temp.append(i)
            elif i is ')':
                if len(temp) == 0:
                    return False
                elif temp[-1] is '(':
                    temp.pop()
                else:
                    temp.append(i)
        pattern = r'^\(*\d+(\.\d+)?((\+|\*|/|-)\(*\d+(\.\d+)?\)*)*(\+|\*|/|-)\d+(\.\d+)?\)*$'
        res = re.match(pattern, strs)
        if len(temp) == 0 and res is not None and res.endpos == len(strs):
            return True
        return False

    def Infix2Prefix(self, infixstrs):
        """
        strs: 中缀式字符串
        return: 转换后的前缀式
        """
        infixstrs = infixstrs.replace(' ','')
        stack1 = []
        stack2 = []

        # 判断算式格式是否正确
        if self.isFormula(infixstrs) is not True:
            return "请输入正确格式的算式！！！"

        infixstrs = infixstrs.replace(' ', '')
        intflag = 0
        decimalflag = 0
        strs = infixstrs[::-1]
        for i in strs:
            if i.isdigit() or self.Operator[i] == 0:
                if i.isdigit() is False and self.Operator[i] == 0:
                    decpart = stack2.pop()
                    for j in range(intflag):
                        decpart = decpart/10
                    stack2.append(decpart)
                    decimalflag = 1
                    intflag = 0
                else:
                    if intflag == 0 and decimalflag == 0:
                        stack2.append(int(i))
                    elif intflag == 0 and decimalflag == 1:
                        stack2.append(stack2.pop() + int(i))
                    elif intflag > 0 :
                        dec = 1
                        for j in range(intflag):
                            dec = 10 * dec
                        stack2.append(stack2.pop() + int(i) * dec)
                    intflag = intflag + 1
            else:
                intflag = 0
                decimalflag = 0
                if len(stack1) == 0  or self.Operator[i] == 3:
                    stack1.append(i)
                elif self.Operator[i] == 4:
                    length = len(stack1)
                    for j in range(length):
                        temp = stack1.pop()
                        if self.Operator[temp] == 3:
                            break
                        stack2.append(temp)
                elif self.Operator[i] == 1 or self.Operator[i] == 2:
                    if self.Operator[stack1[-1]] == 3:
                        stack1.append(i)
                    elif self.Operator[stack1[-1]] <= self.Operator[i] :
                        length = len(stack1)
                        for j in range(length):
                            if len(stack1) == 0 or self.Operator[stack1[-1]] > self.Operator[i]:
                                break
                            temp = stack1.pop()
                            stack2.append(temp)
                        stack1.append(i)
                    else:
                        stack1.append(i)

        while len(stack1)>0:
            stack2.append(stack1.pop())
        result = stack2
        result.reverse()
        return result

    def calculate(self, strs, is_Prefix=True):
        # 根据转换后的式子计算最终的结果
        result = []
        if is_Prefix:
            strs = strs[::-1]
            
        if strs == '请输入正确格式的算式！！！':
            return ['Input error']

        for i in strs :
            if i in self.Operator.keys():
                operandA = result.pop()
                operandB = result.pop()
                if is_Prefix:
                    calres = self.operate(operandA, operandB, i)
                else:
                    calres = self.operate(operandB, operandA, i)
                result.append(calres)
            else:
                result.append(i)
        return result[0]

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

class Paint(object):

    DEFAULT_PEN_SIZE = 9.0  # 画笔的粗细
    DEFAULT_COLOR = 'black' # 画笔的颜色

    def __init__(self):
        self.root = Tk()
        # 设置标题
        self.root.title("手写算式识别与计算")
        self.root.resizable(0,0)

        self.model = Model()

        # 预测按钮
        self.brush_button = Button(self.root, text='  预测  ', command=self.Predict)
        self.brush_button.grid(row=0, column=1)

        # 清屏按钮
        self.eraser_button = Button(self.root, text='  清屏  ', command=self.use_eraser)
        self.eraser_button.grid(row=0, column=2)

        # 解释对象
        self.explain_list = ttk.Combobox(self.root, width=30)
        self.explain_list['value'] = ('选择需要解释的符号')
        self.explain_list.current(0)
        self.explain_list['state'] = 'readonly'
        self.explain_list.configure(font=("Courier", 13))
        self.explain_list.grid(row=0, column=3)

        combostyle = ttk.Style()
        combostyle.theme_create('combostyle', parent='alt',
                                settings={'TCombobox':
                                    {'configure':
                                        {
                                            'foreground': 'black',  # 前景色
                                            'selectbackground': 'black',  # 选择后的背景颜色
                                            'fieldbackground': 'white',  # 下拉框颜色
                                            'background': 'white',  # 下拉按钮颜色
                                        }}}
                                )
        combostyle.theme_use('combostyle')

        # 解释按钮
        self.explain_button = Button(self.root, text='  解释  ', command=self.Explain)
        self.explain_button.grid(row=0, column=4)

        # 绘制算式的画板
        self.c = Canvas(self.root, bg='white', width=850, height=450)
        self.c.grid(row=1, columnspan=5)

        # 预测结果呈现的地方
        self.predictionLabel = Text(self.root, fg='black', height=1, width=20, borderwidth=1, highlightthickness=0, relief='ridge')
        self.predictionLabel.grid(row=0, column=6,padx=15,pady=10,stick=W+E+N+S)
        self.predictionLabel.configure(font=("Courier", 20))

        # CNN解释结果的展示
        self.CNNexplain = Text(self.root, height=7, width=46, borderwidth=0, highlightthickness=0,relief='ridge')
        self.CNNexplain.grid(row=1, column=6,sticky=S+E,pady=15,padx=20)
        self.CNNexplain.configure(font=("Courier", 13))

        # 绘制标注了检测框的图片
        self.image = Canvas(self.root, width=318*(260/160)+125, height=160*(260/160), highlightthickness=0, relief='ridge')
        self.image.create_image(0, 0, anchor=NW, tags="IMG")
        self.image.grid(row=1,column=6,sticky=N,pady=5,padx=15)

        self.nnImageOriginal = Image.open("images/1037.png")
        self.resizeAndSetImage(self.nnImageOriginal)

        # 绘制带有决策依据的图片 
        self.image2 = Canvas(self.root, width=160, height=160, highlightthickness=0, relief='ridge')
        img = Image.open("images/Tensorflow_logo.png")
        img = img.resize((160,160), Image.ANTIALIAS)
        self.photo = ImageTk.PhotoImage(img)
        self.image2.create_image(0, 0, image=self.photo, anchor=NW, tags="IMG2")
        self.image2.grid(row=1,column=6,sticky=W+S,pady=15,padx=15)

        # 作者信息
        self.github = Label(self.root, text="Youjia Zhang", cursor="hand2")
        self.github.bind("<Button-1>", self.openGitHub)
        self.github.grid(row=0, column=0)
        self.github.configure(font=("Courier", 15))

        self.setup()
        self.root.mainloop()

    def resizeAndSetImage(self, image):
        # 修改图片的大小
        NUM = 260
        size = (int(int(float(image.size[0]))/(int(float(image.size[1])/NUM))), NUM)
        resized = image.resize(size, Image.ANTIALIAS)
        self.nnImage = ImageTk.PhotoImage(resized)
        self.image.delete("IMG")
        self.image.create_image(0, 0, image=self.nnImage, anchor=NW, tags="IMG")
    
    def openGitHub(self, event):
        webbrowser.open_new(r"https://github.com/YoujiaZhang")

    def setup(self):
        self.old_x = None
        self.old_y = None
        self.line_width = self.DEFAULT_PEN_SIZE
        self.color = self.DEFAULT_COLOR
        self.eraser_on = False
        self.c.bind('<B1-Motion>', self.paint)
        self.c.bind('<ButtonRelease-1>', self.reset)

    def Predict(self):
        self.c.postscript(file="images/tmp.ps")
        img = Image.open("images/tmp.ps")
        img.save("images/out.png", "png")

        self.predictions, self.order, self.Scores = self.model.Predict("images/out.png",)

        self.predictionLabel.delete(1.0, END)
        # 重新设置图片
        img = Image.open("images/current.png")
        self.resizeAndSetImage(img)

        string = ''
        chars = []
        for p in self.predictions:
            if p > 9:
                if p == 10:
                    string += '+'
                    chars.append('+')
                elif p == 11:
                    string += '-'
                    chars.append('-')
                elif p == 12:
                    string += '×'
                    chars.append('×')
                elif p == 13:
                    string += '÷'
                    chars.append('÷')
                elif p == 14:
                    string += '('
                    chars.append('(')
                elif p == 15:
                    string += ')'
                    chars.append(')')
            else:
                string += str(p)
                chars.append(str(p))
        
        # 设置下拉菜单的选项
        self.explain_list['value'] = tuple(chars)
        self.explain_list.current(0)

        # 初始化 计算单元
        myCalculator = Calculator()

        # 计算识别的算式字符串
        string_result = str(myCalculator.calculate(myCalculator.Infix2Prefix(string)))
        if string_result == '！':
            string_result = 'Input error ~'
        else:
            string_result = string + ' = '+ string_result
        self.predictionLabel.insert(END, string_result)
    
    def Explain(self,):
        # 挑选一个符号进行解释性分析
        p_i = self.explain_list.current()
        splitImage = ImageOps.invert(Image.open("images/split/"+str(self.order[p_i][1])+".png").convert('L'))
        splitImage = np.array(splitImage.resize(size=(28,28)))
        splitImage = splitImage / 255.0
        splitImage = splitImage.reshape(28, 28)
        Explain(splitImage, self.model.model, self.predictions[p_i], self.Scores[p_i], [self.CNNexplain])

        # 重新设置图片
        img = Image.open("images/split/exp.png")
        img = img.resize((160,160), Image.ANTIALIAS)
        self.photo = ImageTk.PhotoImage(img)
        self.image2.delete("IMG2")
        self.image2.create_image(0, 0, image=self.photo, anchor=NW, tags="IMG2")

    def use_eraser(self):
        # 清屏 
        self.predictionLabel.delete(1.0, END)
        self.CNNexplain.delete(1.0, END)
        self.c.delete("all")
        self.resizeAndSetImage(self.nnImageOriginal)
        img = Image.open("images/Tensorflow_logo.png")
        img = img.resize((160,160), Image.ANTIALIAS)
        self.photo = ImageTk.PhotoImage(img)
        self.image2.create_image(0, 0, image=self.photo, anchor=NW, tags="IMG2")
        self.explain_list['value'] = ('选择需要解释的符号')
        self.explain_list.current(0)

    def paint(self, event):
        # 画笔
        self.line_width = self.DEFAULT_PEN_SIZE
        paint_color = 'white' if self.eraser_on else self.color
        if self.old_x and self.old_y:
            self.c.create_line(
                self.old_x, self.old_y, event.x, event.y, 
                width=self.line_width, fill=paint_color,
                capstyle=ROUND, smooth=TRUE, splinesteps=36)
        self.old_x = event.x
        self.old_y = event.y

    def reset(self, event):
        # 重置
        self.old_x, self.old_y = None, None

In [ ]:
ge = Paint()